In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import csv


path = r'C:\Users\tiany\Documents\Columbia University\EE6895 big data\bill\110'
allFiles = glob.glob(path + "/*.csv")
#allFiles = glob.glob("bill/*.csv")
#create lists
#vote same or not between party and individuals

#try legislator 'Neil Abercrombie''Democrate'/ 'Stephanie Herseth Sandlin

dfn = pd.read_csv(allFiles[1],  encoding='latin-1',low_memory=False)
name = dfn.columns.values.tolist()
v = name[3]
dfn = dfn.rename(columns={v:'Vote'})
dfn = dfn.drop_duplicates(subset=['Legislator'])
dfn = dfn['Legislator'].tolist()
proba = []
for u in range(40):
#vote same or not between party and individuals
    Party =[]
    Party_vote = []
    position = []
    selfvote = []
    #contribution differences
    difference = []


    #try legislator 'Neil Abercrombie''Democrate'/ 'Stephanie Herseth Sandlin

    for i in range(len(allFiles)):
        df = pd.read_csv(allFiles[i], encoding='latin-1',low_memory=False)
        #print(allFiles[i])
        df['Contribution Amount'] = df['Contribution Amount'].str.replace('$','')
        df['Contribution Amount'] = df['Contribution Amount'].astype(float)
        name = df.columns.values.tolist()
        v = name[3]
        n = name[-4]
        df = df.rename(columns={n: 'Interest Group Position',v:'Vote'})
        clean = df.drop(['Represents','Contribution Type','Presidential','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        g = clean.groupby(['Party','Interest Group Position']).sum()
        newclean =  df.drop(['Represents','Contribution Type','Presidential','Contribution Amount','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        dd = newclean.drop_duplicates(subset=['Legislator', 'Party', 'Vote'])
        a = (dd['Party'][dd['Vote']=='No']=='Democrat').sum()
        b = (dd['Party'][dd['Vote']=='Not Voting']=='Democrat').sum()
        c = (dd['Party'][dd['Vote']=='Yes']=='Democrat').sum()
        d = (dd['Party'][dd['Vote']=='No']=='Independent').sum()
        e = (dd['Party'][dd['Vote']=='Not Voting']=='Independent').sum()
        f = (dd['Party'][dd['Vote']=='Yes']=='Independent').sum()
        j = (dd['Party'][dd['Vote']=='No']=='Republican').sum()
        k = (dd['Party'][dd['Vote']=='Not Voting']=='Republican').sum()
        l = (dd['Party'][dd['Vote']=='Yes']=='Republican').sum()

        #dataframe
        pd.set_option('display.precision',15)
        pd.DataFrame({'x':[.000000000000001]})

        arrays = [['Democrat', 'Democrat','Democrat', 'Republican','Republican', 'Republican'],
                              ['Oppose','', 'Support', 'Oppose','', 'Support']]
        tuples = list(zip(*arrays))
        index = pd.MultiIndex.from_tuples(tuples, names=['Party', 'Interest Group Position'])
        column = ['Contribution Amount', 'Vote', 'Count']
        s = pd.DataFrame(index=index, columns=column)
        votes = ['No','Not Voting','Yes','No','Not Voting','Yes']
        count = [a,b,c,j,k,l]
        s['Vote'] = votes
        s['Count'] = count
        newframe = s.fillna(g)
        effect = newframe.fillna(0)
        gg = clean.groupby(['Legislator','Party','Vote','Interest Group Position']).sum()
        leglist = newclean.drop_duplicates(subset=['Legislator'])
        leglist = leglist['Legislator'].tolist()
        num = len(leglist)
        reset = gg.reset_index()
        #Tammy Baldwin   'Tom Allen' 'Xavier Becerra'
        try:
            name = reset[reset['Legislator']=='Xavier Becerra']
            if name['Party'].any() == 'Democrat':
                Party.append(1)
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]

                #party veto No
                if a > c:
                    Party_vote.append(0)
                #party veto Yes
                if a < c:
                    Party_vote.append(1)

                #position
                if con_sup > con_oppo:
                    dif = con_sup - con_oppo
                    difference.append(dif)
                    position.append(1)
                if con_sup < con_oppo:
                    dif = con_sup - con_oppo
                    difference.append(dif)
                    position.append(0)
                if con_sup == con_oppo:
                    difference.append(0)
                    position.append(2)

                if name.iloc[0]['Vote'] =='No':
                    selfvote.append(0)
                if name.iloc[0]['Vote'] =='Yes':
                    selfvote.append(1)
                if name.iloc[0]['Vote'] =='Not Voting':
                    selfvote.append(2)
            if name['Party'].any() == 'Republican':
                Party.append(1)
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]

                #party veto No
                if a > c:
                    Party_vote.append(0)
                #party veto Yes
                if a < c:
                    Party_vote.append(1)

                #position
                if con_sup > con_oppo:
                    dif = con_sup - con_oppo
                    difference.append(dif)
                    position.append(1)
                if con_sup < con_oppo:
                    dif = con_sup - con_oppo
                    difference.append(dif)
                    position.append(0)
                if con_sup == con_oppo:
                    difference.append(0)
                    position.append(2)

                if name.iloc[0]['Vote'] =='No':
                    selfvote.append(0)
                if name.iloc[0]['Vote'] =='Yes':
                    selfvote.append(1)
                if name.iloc[0]['Vote'] =='Not Voting':
                    selfvote.append(2)
        except:
            pass

    col = ['Party','Party Vote', 'Group Position','Contribution difference','self Vote']
    mdf = pd.DataFrame(columns=col)    
    mdf['Party'] =  Party
    mdf['Party Vote'] = Party_vote
    mdf['Group Position'] = position
    mdf['Contribution difference'] = difference
    mdf['self Vote'] = selfvote
    mdf['train'] = np.random.uniform(0, 1, len(mdf)) <= .75
    train, test = mdf[mdf['train']==True], mdf[mdf['train']==False]
    #print('Number of observations in the training data:', len(train))
    #print('Number of observations in the test data:',len(test))
    features = mdf.columns[:4]
    y = np.array(train['self Vote'])
    # Create a random forest Classifier. By convention, clf means 'Classifier'
    clf = RandomForestClassifier(n_jobs=2, random_state=0)
    clf.fit(train[features], y)
    predict = clf.predict(test[features])
    og = np.array(test['self Vote'])
    bt=0
    for i in range(len(og)):
        if predict[i] == og[i]:
            bt+=1
    p = bt/len(og)
    proba.append(p)
    #print(dfn[u] +'prob  :',p)
pp = pd.DataFrame(columns=['Legislator','Accuracy'])
pp['Legislator'] = dfn[0:40]
pp['Accuracy'] = proba
pathf = r'C:\Users\tiany\Documents\Columbia University\EE6895 big data\bill\ '
pp.to_csv(pathf + 'final 40 legis accuracy.csv')

C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison

C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)
